## Automatically define shore-normal transects
To be loaded into CoastSat as .geojson

Vague idea: Something with shapely or PostGIS

#### File History
28.09.2022
- Load the extracted shorelines (pkl)
- Remove 'bad' shorelines from the dictionary

#### ToDo

- Compute a mean shoreline --- alternative from Luijendijk (2018): openstreetmap global coastline as reference
- Check the number of segments
- Define lines perpendicular to each segment
    - e.g. in the middle of each segment
    - or with a fixed spacing?

In [1]:
import pickle
from datetime import datetime

In [2]:
datapath = './data/TERS_L8_2018/'

with open(datapath + 'TERS_L8_2018_output.pkl', 'rb') as f:
    sds = pickle.load(f)

### Remove 'bad' shorelines from dict using their dates

try 3, inspired by [this](https://www.geeksforgeeks.org/python-program-to-remove-nth-element-from-kth-keys-value-from-the-dictionary/)

In [3]:
def remove_bad_shorelines(sds, dates_bad_list):
    # round to dates to make them comparable
    dates_bad = [i.date() for i in dates_bad_list] # they call it list comprehension
    dates_all = [i.date() for i in sds["dates"]]
    
    sds_clean = dict()

    for key, val in sds.items():
        sds_clean[key] = (val if key != 'dates' else [idx for idx in val if idx.date() not in dates_bad])
    return sds_clean

In [4]:
dates_bad_list = [datetime(2018,7,10), datetime(2018,9,3)]
sds_clean = remove_bad_shorelines(sds, dates_bad_list)

### Average the extracted shorelines
Did not find a quick solution with shape or PostGIS

Might be an overkill, maybe the reference shoreline or a shoreline from an external dataset (Luijendijk (2018) use OpenStreetMap for this) would be sufficient.

In [10]:
sds_clean.keys()

dict_keys(['dates', 'shorelines', 'filename', 'cloud_cover', 'geoaccuracy', 'idx', 'MNDWI_threshold', 'satname'])

In [13]:
for i in range(0, len(sds_clean['shorelines'])):
    print(len(sds_clean['shorelines'][i]))

1303
1490
1121
1481
1485
1623
1491
1480
1515
1482
744
1480
1125
1461
1474
1426
1319


Number of points per shoreline is not equalized. Therefore no matching pairs of points. Define areas and average the points that fall in one area. The area size influences the resulting segment length. Areas should not be like grid boxes, but rather search circles around the points of one arbitrary shoreline. Search circle too small: Not all shorelines considered. Search circle too large: Segments become large, lower number of transects.

Would maybe be a good idea to first correct the shorelines for tides.